<div
    style='background-image: url("images/microseisms.jpg"); padding: 0px;
    background-size: cover; border-radius: 10px; height: 250px;
    background-position: 50% 50%'>
    <div
        style="float: left; margin: 20px; padding: 10px;
        background: rgba(255 , 255 , 255 , 0.8); width: 85%; height: 150px;
        border-radius: 10px">
        <div
            style="position: relative; top: 50%;
            transform: translatey(-50%)">
            <div
                style="font-size: xx-large; font-weight: 900;
                color: rgba(0 , 0 , 0 , 0.9);
                line-height: 100%">
                Introduction to seismo-acoustic waves in the Earth’s spheres
            </div>
            <div
                style="font-size: large; padding-top: 20px;
                color: rgba(0 , 0 , 0 , 0.7)">
                <p>CTG Course by <em>Läslo Evers, Shahar Shani-Kadmiel,
    and Pieter Smets</em>, CEG, TU Delft
            </div>
        </div>
    </div>
</div>

# Introduction

## The Comprehensive nuclear-Test-Ban Treaty

The Comprehensive Nuclear-Test-Ban Treaty (CTBT) bans nuclear explosions by everyone, everywhere: on the Earth's surface, in the atmosphere, underwater and underground.

![nuke](images/nuke.png)
***Monitoring of (nuclear) explosions in all three mediums***


In [ ]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/w5jgEXgtgEI" frameborder="0" allowfullscreen></iframe>

## The International Monitoring System (IMS)

The verification regime of the CTBT is designed to detect any nuclear explosion conducted on Earth - underground, underwater or in the atmosphere.

![map](images/IMS_map.png)

Arrays may vary in shape and size depending on their purpose. For instance, seismic arrays if the IMS:

![ims seismic arrays](images/IMS_seismic_arrays.png)

and infrasound arrays of the IMS:

![ims infrasound arrays](images/IMS_infrasound_arrays.png)



# Array design and processing theory

**Why arrays?**

* Improve signal-to-noise ratio (SNR)
* Retrieve wavefront parameters such as incoming direction and propagation velocity:

## Background theory: plane wave beamforming

Consider an inclinede three-dimensional planar wavefield $\tau$ with position $\mathbf{x}=(x,y,z)$ at time $t$ travelling over an arbitrary array of four receivers on the horizontal $xy$-plane, i.e., the Earth's surface, with receiver position $\mathbf{r}_n=(x,y)$.
The plane wave is monochromatic (of a single frequency) with a unitary amplitude.

Propagation of the plane wave is described by a three-dimensional wave vector $\mathbf{k}_0$, normal to the wavefront, and is related to the wave slowness vector $\mathbf{p}_0$ by, 

\begin{equation}\mathbf{p}=\omega^{-1}\mathbf{k},\end{equation}

with angular frequency $\omega=2\pi f$.

The wavefront is characterized by wavelength $\lambda$ and a propagation velocity $c$, related to the the magnitude of $\mathbf{k}$ and $\mathbf{p}$ by,

\begin{equation}\lambda = \dfrac{2\pi}{\lVert\vec{k}\rVert} = \dfrac{f^{-1}}{\lVert\vec{p}\rVert}\end{equation}

with $c = \lambda f$.

The local orientation of the three-dimensional wave or slowness vector is defined by the grazing angle $\theta$ with respect to the local horizon and azmuth angle $\phi$ clockwise from north.

![figure 2.1](images/plane_wave_array.png)
![figure 2.1](images/plane_wave_array_cs.png)

The three-dimensional slowness vector denotes,

\begin{equation}
	\mathbf{p} := \begin{pmatrix} p_{x} \\ p_{y} \\ p_{z} \end{pmatrix}
	=  c^{-1} \begin{pmatrix} 
		\cos \theta \ \sin \phi \\
		\cos \theta \ \cos \phi \\
		\sin \theta
	\end{pmatrix},
\end{equation}

The wavefron in the receiver plane propagates with slowness $\mathbf{s}_0$, the projected slowness vector $\mathbf{p}$ onto the two-dimensional receiver $xy$-plane, given by

\begin{equation}
	\mathbf{s} := \begin{pmatrix} s_{x} \\ s_{y} \end{pmatrix}
	= c^{-1}_{\text{app}} \begin{pmatrix} \sin \phi \\ \cos \phi \end{pmatrix}.
\end{equation}

The magnitude of the horizontal slowness $\mathbf{s}$ yields the apparent velocity $c_{\text{app}}$ which is related to the true propagation velocity of the wave $c$ by,
\begin{equation}
	c_{\text{app}} = \frac{c}{\cos \theta}.
\end{equation}
ranging from $c$ up to infinity depending on the incidence angle $\theta$. The origin of propagation is indicated by back azimuth $\phi_b = \phi ± \pi$.

Green lines in the top-view indicate the wavefront at receiver locations $\mathbf{r}_n$ corresponding with delay times $\tau_n$. Delay times in the receiver plane are defined by
\begin{equation}
\tau_n = \mathbf{s}_0 \cdot \mathbf{r}_n
\end{equation}
for a propagating plane wave with respect to the array centre coordinate.

![animated plane wave](images/planewave.gif)


## Array Design and Response

[Open the array design module in a separate notebook](Interactive_array_design_and_response.ipynb)

Start by making an array with a similar design as the one above (3 elements in a triangular configuration with another element in the center)

The array response is a theoretical measure for the best estimate or sensitivity of receivers $\mathbf{r}_n$ to plane wave $\mathbf{s}_0$ with frequency $f_0$ represented by an infinite aperture array regularly sampled in $\mathbf{s}$.
\begin{equation}
	R( \mathbf{s}, \omega ) = 
	\left|
		\dfrac{1}{N} \sum_{n=1}^N e^{-i \omega \left( \mathbf{s}-\mathbf{s}_0 \right) \cdot \mathbf{r}_n }
	\right|^2
\label{eq:receiver:array-response}.
\end{equation}

## Plane wave beamforming

Define the horizontal slowness components of a plane wave given a specific back azimuth and apparent velocity.

The horizontal slowness shall be used to calculate the time differences $\tau_n$ at each receiver position given that specific plane wave at the array centre (0,0).
Hence, the individual receiver traces need to be shifted with $-\tau_n$ to time align all individual recordings to beam steer or _focus_ the array towards the given slowness vector.

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import f as f_stats

In [ ]:
# %load beam_steering_solution
deg2rad = np.pi / 180
def theta_app_vel2pxpy(theta, app_vel):
    """
    Calculate the horizontal slowness vector p(px, py) for a propagating
    plane wave with back azimuth `theta` and horizontal propagation
    velocity (apparent velocity) `app_vel`.
    
    Parameters
    ----------
    theta : float
        Back azimuth in degrees clockwise from North.
    
    app_vel : float
        Horizontal propagation velocity in m/s.
        
    Returns
    -------
    px, py : float
        Horizontal components of the slowness vector p.
    """
    # convert theta to radians
    theta *= deg2rad
    
    # write an expression for the px, and py slowness vectors
    px = ...
    py = ...

    return px, py


def get_sample_shifts(x, y, px, py, samp_rate=20):
    """
    Calculate the integer sample shifts needed to align the traces for a
    given slowness vector `p` describing a propagating plane wave across
    an array with configuration `r_n(x_n, y_n)`.
    
    Parameters
    ----------
    x, y : array-like
        x and y coordinates (m) of the array elements (`n`-elements).
        
        ..note:: x, y coordinates should be detrended. That is, offsets
                 relative to array center such that
                 x.mean() = y.mean() = 0.
        
    px, py : float
        Horizontal components of the slowness vector p (s/m).

    samp_rate : float
        Sampling rate of the data in samples per second (Hz).
    
    Returns
    -------
    sample_shifts : array-like
        Number of samples needed to be added (shifted) to align traces
        with the center.
    """
    # center the array as all parameters are relative to array center.
    x, y = np.array((x, y))
    
    if not np.allclose(np.array((x.mean(), y.mean())), np.zeros(2)):
        raise ValueError('Array is not centered at O(0,0). Detrend x, y')
    
    sample_shifts = -(
        samp_rate * (
            x * px + y * py
        ) + 0.5  # adding 0.5 ensures values are rounded to the nearest int
    ).astype(np.int64)
    return sample_shifts

Copy the x and y coordinates from your array design and find out what are the number of samples needed to shift each trace to time-align a signal arriving from direction $\theta$ and propagates with apparent velocity $c_{app}$.

In [ ]:
# paste coordinates here:
x = ...
y = ...


fig, ax = plt.subplots()
ax.set_aspect(1)
r_max = np.abs(np.hstack((x, y))).max() * 1.2
ax.axis([-r_max, r_max] * 2)

theta = 30
app_vel = 340
sample_shifts = get_sample_shifts(x, y, *theta_app_vel2pxpy(theta, app_vel))
for x_, y_, shift in zip(x, y, sample_shifts):
    ax.plot(x_, y_, 'ko')
    ax.text(x_, y_ + 50, '{} samples'.format(shift),
            ha='center', va='bottom')
    
x1, y1 = r_max * np.array((np.sin(theta * deg2rad), np.cos(theta * deg2rad)))
ax.annotate('', (x1, y1), xytext=(0, 0),
            arrowprops=dict(arrowstyle='-|>', color='r'))

ax.grid(True, which='both', c='0.9', lw=0.5, zorder=0)

ax.set_xlabel('x, m')
ax.set_ylabel('y, m')

ax.set_title('Array configuration')


## Fisher statistics
The Fisher ratio is a metric to evaluate the signal coherency of a series of traces as a whole (thus an array). The Fisher ratio is used to evaluate the time shifted traces. It is based on [Melton and Bailey (1957), Multiple signal correlators, Geophysics](https://library.seg.org/doi/10.1190/1.1438390).

Have a look at the article and reproduce the given example using Tables 1 and 2 (page 573 in the article). The tables are already given below. You need to complete apply the correlation algorithm given on page 576 and explaned in the steps above (pages 574 to 577).

In [ ]:
MB57_table_1 = np.array([
    [6,23,14,27,23],  # trace a
    [12,37,7,38,34],  # trace b ...
    [46,19,24,4,27],
    [23,32,18,47,14],
    [29,25,36,43,28]  # trace e
], np.int32)

MB57_table_2 = MB57_table_1.copy()

# add 12 to column 2 & 5
MB57_table_2[:,1] +=  12
MB57_table_2[:,4] += -12

# N: number of array elements (traces), T: number of samples
N, T = MB57_table_1.shape

Plot the two sets of traces.

In [ ]:
# plot
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(7, 3), sharey=True)
fig.subplots_adjust(bottom=0.2)

for i, trace in enumerate(MB57_table_1):
    line, = ax1.plot(trace, label=chr(65+i))

ax1.set_xlabel('sample')
ax1.set_ylabel('value')

for i, trace in enumerate(MB57_table_2):
    line, = ax2.plot(trace, label=chr(65+i))

ax2.set_xlabel('sample')

# Put a legend below current axis
ax2.legend(loc='upper center', bbox_to_anchor=(0, -0.15),
           ncol=N, frameon=False)

### Exercise:

Write a function to evaluate the Fisher ratio.

what is the Fisher value of tables 1 and 2?


In [ ]:
# %load fratio_solution
def compute_fratio(win):
    """
    Function to calculate the Fisher ratio based on Melton and Bailey
    [1957].
    
    Parameters
    ----------
    win : array-like
        2D array with N traces (rows) of length T samples (cols).
        
    Returns
    -------
    fratio : float
        The Fisher ratio value.
    """
    N, T = win.shape
    fratio = ...
    return fratio

def fratio2snr(fratio, N):
    return np.sqrt((fratio - 1) / N)


def snr2fratio(snr, N):
    return snr**2 * N + 1


def fratio_significance(N, T, alpha=0.05):
    return f_stats.ppf(1 - alpha, T - 1, T * (N - 1))


F-ratio table 1 should be 0.654.

In [ ]:
compute_fratio(MB57_table_1)

F-ratio table 2 should be 3.386.

In [ ]:
compute_fratio(MB57_table_2)

You can check the significance of the F-ratio given the number of receivers $N$ and time samples $T$.

In [ ]:
fratio_significance(N,T,0.05)

The Fisher ratio is directly related to the signal-to-noise ratio (SNR) of the array:

In [ ]:
fratio2snr(compute_fratio(MB57_table_2),N)

## Why Fisher ratio?

There are many beamforming algorithms. They are divided into the forward and  inverse approach. In the inverse approach, a pair-wise cross-correlation method is used to resolve the lag time that provides the maximum correlation. This is an expensive operation from a computational point of view:

\begin{equation}
    \frac{N!}{2(N-2)!} \cdot 2T-1
\end{equation}

For all pair combinations in an 8-element array with a window size of 400 seconds this amounts to 28 (combinations) x 798 (samples) = 22,344 operations (per window).

This will resolve one lag time per element pair and then a wave needs to be fitted to the lag times between all elements. In case of low SNR this can result in inaccurate lag times. Furthermore, as this method uses pairs of sensors, a strongly correlated signal (or noise) between two elements can introduce an error to the plane wave solution.

In the forward approach, plane waves are predefined by a set of slowness vectors. Time shifts are pre-calculated with regard to the array geometry and sampling rate.

For each slowness vector the Fisher ratio is evaluated. The maximum value corresponds to the slowness of the most coherent plane wave. The back azimuth and apparent velocity can be retrieved from this slowness. The bestbeam, is then constructed by delay and sum of the traces in each time window.

As the Fisher ratio can be evaluated over the entire grid, one can resolve not only the global maximum but also local maxima which may reveal multiple sources. A word of caution should be mentioned here with regard to side-lobes which you saw in the array design and response demo.


In case of low SNR or multiple sources, the forward approach using an F-detector out performs the cross-correlation methods.

In [2]:
from timefisher import beamform, _do_loops_python
_do_loops_python??

In [ ]:
from grid import Grid
grid = Grid(app_vel_params=(200, 50))

plt.close('all')
grid.plot_pxpy()

## Example

In [ ]:
# get the data and array metadata (inventory)

from obspy import UTCDateTime
from obspy.clients.fdsn import Client as fdsnClient

# Plofkraak @ CIA
# sta = 'CIA*'
# t0  = obspy.UTCDateTime("2019-02-14T02:20:00.000Z");
# t1  = obspy.UTCDateTime("2019-02-14T02:35:00.000Z");

# Perseids meteor shower @ DBN
sta = 'DBN*'
t0  = UTCDateTime("2018-08-13T05:00:00.000Z");
t1  = UTCDateTime("2018-08-13T05:15:00.000Z");

# get StationXML
client = fdsnClient('orfeus')
net = 'NL'
cha = 'HDF'

inv = client.get_stations(
    network=net,
    station=sta,
    channel=cha,
    starttime=t0,
    endtime=t1,
    level='response'
)
print(inv)

inv.write(
    '../../Data/DBN.xml', format='StationXML'
)

In [ ]:
try:
    stream = client.get_waveforms(net, sta, '*', cha, t0, t1).sort()
except:
    raise ValueError('No data found for {}.{}.{}.{}.*'.format(net, sta, '*', cha))
    
stream.write(
    '../../Data/DBN.20180813050000.mseed', format='MSEED'
)

stream.merge(fill_value=0)
filter_params = (0.5, 1, 20, 25)
stream.remove_response(inv, pre_filt=filter_params)

fig, ax = plt.subplots(stream.count(), sharex=True, sharey=True,
                       figsize=(7, 1 * stream.count()))
fig.subplots_adjust(top=0.95, hspace=0)

for i, tr in enumerate(stream):
    axi = ax[i]
    axi.plot(tr.times(), tr.data, 'k', lw=0.5)
    axi.text(0.95, 0.95, tr.id, ha='right', va='top', transform=axi.transAxes)
    axi.grid(True, axis='x')
    
axi.set_xlabel('time since {}, seconds'.format(t0.strftime('%FT%T')))
axi.set_ylabel('pressure, Pa')

In [ ]:
from ipywidgets import interactive, fixed, Layout, HBox, VBox
import ipywidgets as widgets

from array_response_widget import Array

array = Array(inv)
array.plot_array_response(f_min=3, f_max=3)

out = widgets.Output()

cmaps = plt.colormaps()
w = interactive(
    array.__update_widget__,
    
    c_app=widgets.IntSlider(
        value=280, min=200, max=3000, step=100, continuous_update=False,
        description='C min, m/s'
    ),
    c_steps=widgets.IntSlider(
        value=50, min=10, max=150, step=10, continuous_update=False,
        description='steps', layout=Layout(width='250px')
    ),
    
    fminmax=widgets.FloatRangeSlider(
        value=[3, 3], min=0.05, max=20, step=0.01,
        continuous_update=False, description='Frequency range, Hz',
        style={'description_width': 'initial'}
    ),
    f_steps=widgets.IntSlider(
        value=1, min=1, max=50, step=1, continuous_update=False,
        description='steps', layout=Layout(width='250px')
    ),
    
    log=widgets.Checkbox(
        value=False, description='Log scale'),
    cmap=widgets.Dropdown(
        value='inferno_r', options=cmaps, description='cmap',
                              layout=Layout(width='180px')),
)

center_b = widgets.Button(
    description='Center array', button_style='info', layout=Layout(width='100px')
)
center_b.on_click(array.__center__)

reset_b = widgets.Button(
    description='Reset', button_style='danger', layout=Layout(width='100px')
)
reset_b.on_click(array.__reset__)

controls = VBox([out,
                 HBox([center_b]),
                 HBox(w.children[:2]),
                 HBox(w.children[2:4]),
                 HBox(w.children[4:]),
                 HBox([reset_b])])
controls

### Preparing the data

We have already detrended, tapered, filtered, and deconvolved the instrument response. We need to make sure that all traces begin and end in the same time and have the same number of samples, and add the x, y attributes to each trace, the coordinates of the corresponding array element.

In [ ]:
# trim to the same start and end time
stream.trim(t0, t1, pad=True, fill_value=0)

# assign the x, y attributes and validate smapling rate and number of samples
samp_rate = []
npts = []
for i, tr, in enumerate(stream):
    tr.x, tr.y = array.x[i], array.y[i]
    samp_rate.append(tr.stats.sampling_rate)
    npts.append(tr.stats.npts)
    
if np.any(np.array(samp_rate) - tr.stats.sampling_rate):
    raise RuntimeError("Sampling rate does not match. {}".format(samp_rate))
if np.any(np.array(npts) - tr.stats.npts):
    raise RuntimeError("Number of samples does not match. {}".format(npts))

In [ ]:
from timefisher import beamform, fratio2snr, plot_results
(bestbeam, times, fratio_max,
 baz, app_vel, fgrid) = beamform(stream, grid, 10, 0.9)

snr = fratio2snr(fratio_max, stream.count())

In [ ]:
fig, ax, cb1, cb2 = plot_results(
    bestbeam, times, fratio_max, baz, app_vel, snr, stream, utctime=True
)

ax[0].set_ylim(3, 20)
ax[0].images[0].set_clim(1e-4, 1e-3)